# Broadcasting

Broadcasting refers to the transformation of arrays of non-similar dimensions involved in arithmetic operations. If certain constraints are met (the absence of which would not allow arrays to mathematically be broadcast), the smaller array is broadcast across the larger array such that the new dimensions make arithmetic operations involving the arrays possible. 

We can demonstrate this using Numpy which also allows broadcasting - 

In [1]:
import numpy as np
a = np.array([1.0, 2.0, 3.0])
b = np.array([2.0, 2.0, 2.0])
a*b

array([2., 4., 6.])

This is a simple operation where elements at the ith index of a are multiplied with the elements at the ith index of b. 

Now let us change the value of b to a scalar and then multiply the two - 

In [2]:
b = 2.0 # Redefining b
a*b

array([2., 4., 6.])

Both give the same result! <br>
Here we see that b has been broadcast to represent `[2.0, 2.0, 2.0]` and b has the same shape as a, so that the multiplication operation succeeds.
<img src="images/simplenumpybroadcast.png">

The Awkward Array library (and Numpy) is smart enough to be able to do this without copying the data multiple times to avoid being memory expensive. <Can it?>

### Arrays with Regular Dimensions
*(paraphrasing the Numpy broadcasting documentation)*

For arrays which have regular dimensions, such as Numpy arrays, there are some guidelines set by Numpy which the Awkward Array complies with for when an array can be broadcast.

The fundamental rule is - 

>**The Broadcasting Rule <br>
>In order to broadcast, the size of the trailing axes for both arrays in an operation must either be the same size or one of them must be one.**

Arrays do not need to have the same number of dimensions. For example, if you have a 256x256x3 array of RGB values, and you want to scale each color in the image by a different value, you can multiply the image by a one-dimensional array with 3 values. Lining up the sizes of the trailing axes of these arrays according to the broadcast rules, shows that they are compatible:

```
Image  (3d array): 256 x 256 x 3
Scale  (1d array):             3
Result (3d array): 256 x 256 x 3
```

When either of the dimensions compared is one, the other is used. In other words, dimensions with size 1 are stretched or “copied” to match the other.

In the following example, both the A and B arrays have axes with length one that are expanded to a larger size during the broadcast operation:

```
A      (4d array):  8 x 1 x 6 x 1
B      (3d array):      7 x 1 x 5
Result (4d array):  8 x 7 x 6 x 5
```

Here are some more examples:

```
A      (2d array):  5 x 4
B      (1d array):      1
Result (2d array):  5 x 4

A      (2d array):  5 x 4
B      (1d array):      4
Result (2d array):  5 x 4

A      (3d array):  15 x 3 x 5
B      (3d array):  15 x 1 x 5
Result (3d array):  15 x 3 x 5

A      (3d array):  15 x 3 x 5
B      (2d array):       3 x 5
Result (3d array):  15 x 3 x 5

A      (3d array):  15 x 3 x 5
B      (2d array):       3 x 1
Result (3d array):  15 x 3 x 5
```

<img src="images/numpybroadcastworks.png">

Here are examples of shapes that do not broadcast:

```
A      (1d array):  3
B      (1d array):  4 # trailing dimensions do not match

A      (2d array):      2 x 1
B      (3d array):  8 x 4 x 3 # second from last dimensions mismatched
```

<img src="images/numpybroadcastfails.png">

An example of broadcasting in practice:

In [3]:
x = np.arange(4)
xx = x.reshape(4, 1)
y = np.ones(5)
z = np.ones((3,4))

x.shape

(4,)

In [4]:
y.shape

(5,)

In [5]:
x + y # Cannot broadcast

ValueError: operands could not be broadcast together with shapes (4,) (5,) 

In [6]:
xx.shape

(4, 1)

In [7]:
(xx + y).shape

(4, 5)

In [8]:
xx + y

array([[1., 1., 1., 1., 1.],
       [2., 2., 2., 2., 2.],
       [3., 3., 3., 3., 3.],
       [4., 4., 4., 4., 4.]])

z.shape

In [9]:
(x + z).shape

(3, 4)

In [10]:
x + z

array([[1., 2., 3., 4.],
       [1., 2., 3., 4.],
       [1., 2., 3., 4.]])

<Insert Awkward translation of example?>

<Fixed dimension array broadcasting in other languages?>

### Arrays with Variable Dimensions

You might have noticed that in the previous section, all the arrays have been aligned to the right when broadcasting. This is the behaviour adopted by Numpy and also by Awkward Array for arrays with regular dimensions.

But for arrays with variable dimensions, such as most arrays handled by the Awkward Array library (not possible in Numpy), arrays are aligned to the left when broadcasting. This seems more natural for tree like structures (as found commonly in Physics data) where one would expect the roots of the data structures to line up and allow leaves of the smaller array to be duplicated to allow arithmetic operations with the larger array. 

A simple example - 

In [11]:
import awkward1 as ak
ak.broadcast_arrays([            100,   200,        300],
                    [[1.1, 2.2, 3.3],    [], [4.4, 5.5]])

[<Array [[100, 100, 100], [], [300, 300]] type='3 * var * int64'>,
 <Array [[1.1, 2.2, 3.3], [], [4.4, 5.5]] type='3 * var * float64'>]

One typically wants single-item-per-element data to be duplicated to match multiple-items-per-element data. Operations on the broadcasted arrays like

`one_dimensional + nested_lists`

would then have the same effect as the procedural code

```
for x, outer in zip(one_dimensional, nested_lists):
    output = []
    for inner in outer:
        output.append(x + inner)
    yield output
```
where `x` has the same value for each `inner` in the inner loop.

Things get trickier when we have variable length arrays, 

Just because 2 arrays have the same Awkward Array `shape` and `Array.type`, does not mean that the smaller array can be broadcast to the larger array.

In [12]:
a = ak.Array([1, [2, 3, 4, 5]])
b = ak.Array([1, [2, 3, 4]])

In [13]:
a.shape

(2,)

In [14]:
a.type

2 * union[int64, var * int64]

In [15]:
b.shape

(2,)

In [16]:
b.type

2 * union[int64, var * int64]

In [17]:
# Does not work
ak.broadcast_arrays(b, a)

ValueError: in ListArray64, cannot broadcast nested list

If one were to intuitively think about this, it would be obvious why the broadcasting does not work. How would one extend the smaller array `b` to match the larger array `a`? <br>
<Mathematical explanation?>

<Visualization of above example?>

<Jagged arrays in Java?>

## Further reading - 
* https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html  
* https://docs.scipy.org/doc/numpy/user/theory.broadcasting.html  
* https://awkward-array.readthedocs.io/en/latest/_auto/ak.broadcast_arrays.html  